# MPC Part 2- TP2 : Feature selection and non-linear regression 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# Load the data
ozone = pd.read_csv('ozone.txt', sep = ' ')
ozone_new = pd.read_csv('ozone_n.txt', sep = ' ')
ozone = ozone.append(ozone_new, ignore_index = True)
ozone
# Same dataset than TP1

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10
0,87,15.6,18.5,18.4,4,4,8,0.6946,-1.7101,-0.6946,84
1,82,17.0,18.4,17.7,5,5,7,-4.3301,-4.0000,-3.0000,87
2,92,15.3,17.6,19.5,2,5,4,2.9544,1.8794,0.5209,82
3,114,16.2,19.7,22.5,1,1,0,0.9848,0.3473,-0.1736,92
4,94,17.4,20.5,20.4,8,8,7,-0.5000,-2.9544,-4.3301,114
...,...,...,...,...,...,...,...,...,...,...,...
106,59,18.3,18.3,19.0,7,7,7,-3.9392,-1.9284,-1.7101,66
107,70,17.1,18.2,18.0,7,7,7,-4.3301,-7.8785,-5.1962,72
108,81,19.6,25.1,27.2,3,4,4,-1.9284,-2.5712,-4.3301,57
109,146,27.0,32.7,33.7,0,0,0,2.9544,6.5778,4.3301,121


In [248]:
# Train / Test split:
from sklearn.model_selection import train_test_split
train , test = train_test_split(ozone, test_size = 0.20, random_state = 2) 
# to have the same split than me

## Variable selection

## Polynomial regression

In [265]:
def add_polynomial_feature(data, idx_p, power):
    new_data = data.copy(deep = True)
    for i in range(0, len(idx_p)):
        for j in power:
            for k in range(2, j+1):
                new_data['{}_pow_{}'.format(new_data.columns[idx_p[i]],k)] = new_data.iloc[:,idx_p[i]]**k
    return(new_data)

In [334]:
train_poly = add_polynomial_feature(train, [8], [2])
print(train_poly)
# You can see a new column with values of x8^2
test_poly = add_polynomial_feature(test, [8], [2])
print(test_poly)
# Same for test

       y    x1    x2    x3  x4  x5  x6      x7      x8      x9  x10   x8_pow_2
108   81  19.6  25.1  27.2   3   4   4 -1.9284 -2.5712 -4.3301   57   6.611069
97    77  16.2  20.8  22.1   6   5   5 -0.6946 -2.0000 -1.3681   71   4.000000
5     80  17.7  19.8  18.3   6   6   7 -5.6382 -5.0000 -6.0000   94  25.000000
78    59  16.5  20.3  20.3   5   7   6 -4.3301 -5.3623 -4.5000   76  28.754261
11    72  18.3  19.6  19.4   7   5   6 -0.8682 -2.7362 -6.8944   90   7.486790
..   ...   ...   ...   ...  ..  ..  ..     ...     ...     ...  ...        ...
43    81  18.8  22.5  23.9   6   3   2  0.5209 -1.0000 -2.0000   72   1.000000
22    67  19.5  23.4  23.7   5   5   4 -1.5321 -3.0642 -0.8682   81   9.389322
72   159  25.0  33.5  35.5   1   1   1  1.0000  0.6946 -1.7101  166   0.482469
15    81  16.2  22.4  23.4   8   3   1  0.0000  0.3473 -2.5712  145   0.120617
40    88  16.9  20.3  20.7   6   6   5 -2.8191 -3.4641 -3.0000   92  11.999989

[88 rows x 12 columns]
       y    x1    x2    x3  